In [1]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [20]:
from langchain_gigachat.chat_models import GigaChat
import pickle
from rich.markdown import Markdown
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers.string import StrOutputParser

In [7]:
llm = GigaChat(
    model="GigaChat-2-Max",
    verify_ssl_certs=False,
    profanity_check=False
)

llm.invoke("hello")

AIMessage(content='Привет!', additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 17, 'completion_tokens': 5, 'total_tokens': 22, 'precached_prompt_tokens': 3800}, 'model_name': 'GigaChat-2-Max:2.0.29.04', 'x_headers': {'x-request-id': '65245433-3d93-45e0-bc1c-86c960dad56b', 'x-session-id': '1c50efea-8c44-49d5-9cd6-987746312add', 'x-client-id': None}, 'finish_reason': 'stop'}, id='65245433-3d93-45e0-bc1c-86c960dad56b', usage_metadata={'output_tokens': 5, 'input_tokens': 17, 'total_tokens': 22, 'input_token_details': {'cache_read': 3800}})

In [8]:
with open('../data/annotations/annotations_2025-07-03.pkl', 'rb') as file_:
    annotations_ = pickle.load(file_)
annotations_[0]

{'index': 0,
 'example': {'id': '09851372-b1d1-47f5-9451-3002038c1b3b',
  'name': 'LangGraph',
  'start_time': '2025-07-03 12:36:25.867023',
  'run_type': 'chain',
  'end_time': '2025-07-03 12:42:08.212201',
  'extra': {'inputs_is_truthy': True,
   'runtime': {'sdk': 'langsmith-py',
    'sdk_version': '0.3.44',
    'library': 'langsmith',
    'platform': 'Windows-10-10.0.26100-SP0',
    'runtime': 'python',
    'py_implementation': 'CPython',
    'runtime_version': '3.11.11',
    'langchain_version': '0.3.25',
    'langchain_core_version': '0.3.63'},
   'metadata': {'revision_id': 'c4aea2a-dirty', 'ls_run_depth': 0}},
  'error': None,
  'serialized': None,
  'events': [{'name': 'start', 'time': '2025-07-03T12:36:25.867023+00:00'},
   {'name': 'end', 'time': '2025-07-03T12:42:08.212201+00:00'}],
  'inputs': {'task': {'max_sections': 3,
    'publish_formats': {'markdown': True, 'pdf': True, 'docx': True},
    'include_human_feedback': False,
    'follow_guidelines': False,
    'model': '

In [9]:
errors = []
for annotation in annotations_:
    errors.extend(annotation['_notes'].split('\n'))
len(errors)

64

## Using LLM

In [18]:
errors_str = ' '.join(errors)

In [21]:
system = """Ты — опытный аналитик, который помогает классифицировать и анализировать ошибки на основе кратких описаний.  

**Задача:**  
1. Проанализируй предоставленные заметки об ошибках.  
2. Сгруппируй похожие ошибки, дав каждой группе четкое и понятное название (например, "Ошибка аутентификации", "Проблемы с загрузкой данных", "Некорректное отображение UI").  
3. Составь отчет в следующем формате:  
 
## [Название ошибки]
[Четкое её Описание]
## [Название ошибки]
[Четкое её Описание]
...  
"""

prompt = ChatPromptTemplate.from_messages([("system", system), ("user", "{user_input}")])

In [22]:
chain = prompt | llm | StrOutputParser()

In [23]:
Markdown(chain.invoke({"user_input": errors_str}))

Проблемы с содержанием текста                                           

Отсутствует соответствие между оглавлением и фактическим текстом документа ("Есть заголовок в содержании, но его   
нет в тексте остальном").                                                                                          


                                             Повторения и избыточность                                             

Несколько раз повторяются одинаковые мысли разными словами ("Много раз одну и ту же инфо пишет, просто другими     
словами").                                                                                                         


                                    Некорректное оформление ссылок и источников                                    

Ссылки ведут на несуществующие ресурсы или содержат неверную информацию ("По ссылке нет контента, открывается      
страница, что нет такой страницы", "Год у источников выдумывает", "ссылки на несуществующий ресурс", "где-то с     
сайтов взяли прямую рекламу конкретных компаний, что мб не ок").                                                   


                                   Отсутствие подтверждений фактов и утверждений                                   

Утверждения не подкреплены достоверными источниками ("Встречаются громкие заявления не подтвержденные источниками",
"факты, не подтвержденные источниками", "инфо вообще не подтвержденная источниками").                              


                                      Стилистические и грамматические ошибки                                       

Имеется ряд стилистических ошибок и плеоназмов ("Грамматическая ошибка – 'создавая как новые рабочие места, так и  
приводя к исчезновению традиционных профессий'", "трудовая карьера – плеоназм").                                   


                                   Ошибки оформления структуры и форматирования                                    

Повторяется одно и то же название раздела ("слово заключение два раза написано"), присутствуют проблемы с          
оформлением markdown-разметки ("не везде маркдаун полностью корректен, из-за этого в отчёте встречаются **").      


                                      Некачественные и некорректные источники                                      

Используются некачественные и недостоверные сайты ("самие источники сомнительного качества", "sources example      
domain", "очень общие ссылки, на сайте много статей, непонятно, на какую именно ссылаются").                       


                            Неправильное использование цитат и библиографического стиля                            

Цитаты оформляются неправильно ("цитаты в тексте называют просто – in-text citation"), нарушены требования         
заданного формата оформления списка литературы ("источники в другом стиле представлены, не в заявленном").         


                                  Логически необоснованное объединение материалов                                  

Необоснованно объединены разнородные данные ("сомнительное объединение двух таблиц в одну – профессии старые и     
новые, но в одной строке просто две рандомные, не связанные между собой профессии").

## Using Clusterization

In [24]:
from bertopic.backend import BaseEmbedder
from langchain_gigachat.embeddings import GigaChatEmbeddings

class CustomEmbedder(BaseEmbedder):
    def __init__(self, embedding_model):
        super().__init__()
        self.embedding_model = embedding_model

    def embed(self, documents, verbose=False):
        embeddings = self.embedding_model.embed_documents(documents)
        return embeddings 

# Create custom backend
emb_m = GigaChatEmbeddings(model="EmbeddingsGigaR", verify_ssl_certs=False)
custom_embedder = CustomEmbedder(embedding_model=emb_m)

In [25]:
from umap import UMAP

umap_model = UMAP(n_neighbors=3, n_components=3, min_dist=0.0, metric='cosine')

In [26]:
from hdbscan import HDBSCAN

hdbscan_model = HDBSCAN(min_cluster_size=2, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

In [27]:
from bertopic import BERTopic
import numpy as np

topic_model = BERTopic(embedding_model=custom_embedder, calculate_probabilities=True, verbose=True, hdbscan_model=hdbscan_model, umap_model=umap_model)
embeddings = custom_embedder.embed(errors)
embeddings = np.array(embeddings)

In [28]:
topics, probs = topic_model.fit_transform(errors, embeddings)

2025-08-04 11:44:07,610 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-08-04 11:44:20,724 - BERTopic - Dimensionality - Completed ✓
2025-08-04 11:44:20,726 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-08-04 11:44:20,776 - BERTopic - Cluster - Completed ✓
2025-08-04 11:44:20,791 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-08-04 11:44:20,825 - BERTopic - Representation - Completed ✓


In [29]:
freq = topic_model.get_topic_info(); freq

,Topic,Count,Name,Representation,Representative_Docs
0,0,21,0____,"[, , , , , , , , , ]","[, , ]"
1,1,6,1_источниками_вообще_подтвержденная_подтвержде...,"[источниками, вообще, подтвержденная, подтверж...",[Встречаются громкие заявления не подтвержденн...
2,2,6,2_же_раз_повторяет_вода,"[же, раз, повторяет, вода, инфу, ту, другими, ...","[Несколько раз одни и те же мысли повторяет;, ..."
3,3,5,3_заключение_два_раза_слово,"[заключение, два, раза, слово, написано, дважд...","[слово заключение два раза написано;, слово за..."
4,4,5,4_нет_ссылки_domain_example,"[нет, ссылки, domain, example, конкретных, нес...","[ссылки на несуществующий ресурс;, По ссылке н..."
5,5,4,5_text_in_citation_называют,"[text, in, citation, называют, цитаты, тексте,...",[цитаты в тексте называют просто - in-text cit...
6,6,4,6_новые_приводя_места_как,"[новые, приводя, места, как, рабочие, професси...",[сомнительное объединение двух таблиц в одну -...
7,7,4,7_общие_на_очень_много,"[общие, на, очень, много, запятую, какую, комм...",[просто перечислил сразу кучу источников чере...
8,8,3,8_заголовок_информацию_его_есть,"[заголовок, информацию, его, есть, качества, и...","[сами источники сомнительного качества;, источ..."
9,9,3,9_bigtranstour_2025_accessed_at,"[bigtranstour, 2025, accessed, at, 03, availab...","[Год у источников выдумывает;, источники в дру..."


In [30]:
topic_model.get_topic(1)

[('источниками', np.float64(0.3496807322917733)),
 ('вообще', np.float64(0.19739316420203232)),
 ('подтвержденная', np.float64(0.19739316420203232)),
 ('подтвержденные', np.float64(0.19739316420203232)),
 ('не', np.float64(0.19540585010500372)),
 ('инфа', np.float64(0.15342231389822994)),
 ('громкие', np.float64(0.12159863877986764)),
 ('заявления', np.float64(0.12159863877986764)),
 ('подтвержденных', np.float64(0.12159863877986764)),
 ('факты', np.float64(0.12159863877986764))]

In [31]:
topic_model.visualize_topics()

In [32]:
topic_model.visualize_hierarchy(top_n_topics=50)

In [33]:
topic_model.visualize_heatmap()